# 1. Bacteria Influencing Corrosion
This notebook aims to identify microorganisms that have a recognized influence on corrosion damage. The analysis involves comparing bacteria against known corrosion-related gene sequences and metabolic pathways associated with Microbiologically Influenced Corrosion (MIC).
__Aims__
Search literature on the study selected genera that have been reported as causing corrosion damage, using different terms. Make a comprehensive tabel of the results. Comprehensive search on specific functional genes involved in corrosion processes, focusing on: Sulfate reduction pathways (dsrAB, aprAB genes),metal reduction genes and cytochrome c3 complexes. 
Perform targeted analysis between known corrosion-causing bacteria and newly identified bacterial specimens

__Databases Used__:
    * KEGG (Kyoto Encyclopedia of Genes and Genomes): https://www.genome.jp/kegg/pathway.html.Used for metabolic pathway identification and functional gene annotations  
    * PubMed: Used for literature analysis and validation. 
__Analysis Workflow__    
1. Initial Computational Screening → Search KEGG database for pathways and genes →Literature validation through PubMed 
2. Results Analysis and Documentation → Compilation of findings in Excel sheets → Documentation of references and abstracts
Notebook files
Copy/home/beatriz/MIC/2_Micro/data_Ref/
├── bacteria_corrosion_summary_{timestamp}.xlsx    # Results file for each run
│   ├── Analysis_{timestamp}    # Main results sheet
│   ├── References             # APA formatted references
│   └── Abstracts             # Related paper abstracts
└── Original_data/            # Raw data storage

In [1]:
'''import os
from google.colab import drive  #silence for vscode
drive.mount('/content/drive')
#change the path
os.chdir('/content/drive/My Drive/MIC')
# For colab
!pip install pandas numpy biopython
!pip install requests beautifulsoup4
!pip install Bio'''

"import os\nfrom google.colab import drive  #silence for vscode\ndrive.mount('/content/drive')\n#change the path\nos.chdir('/content/drive/My Drive/MIC')\n# For colab\n!pip install pandas numpy biopython\n!pip install requests beautifulsoup4\n!pip install Bio"

# 2. Preparing data

In [2]:
import os
from pathlib import Path
from Bio import Entrez
import pandas as pd
from functools import partial
import requests
from bs4 import BeautifulSoup
import time
import urllib3
from datetime import datetime
import logging
import numpy as np
import matplotlib.pyplot as plt
import openpyxl
from openpyxl.styles import Alignment
import gc #clutter
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from scholarly import scholarly  # For Google Scholar
from crossref.restful import Works  # For CrossRef

In [3]:
# For VSCode
base_dir = Path("/home/beatriz/MIC/2_Micro/data_Ref")
original_dir = base_dir / "Original_data"
results_file = base_dir / "bacteria_corrosion_summary.xlsx" 

# For Colab
'''
from google.colab import drive
drive.mount('/content/drive')
base_dir = Path('/content/drive/My Drive/MIC/data')
original_dir = base_dir / "original"
original_dir.mkdir(exist_ok=True)
'''

'\nfrom google.colab import drive\ndrive.mount(\'/content/drive\')\nbase_dir = Path(\'/content/drive/My Drive/MIC/data\')\noriginal_dir = base_dir / "original"\noriginal_dir.mkdir(exist_ok=True)\n'

In [4]:
# Read the Excel file for the whole data
Jointax = pd.read_excel("data/Jointax.xlsx", sheet_name='Biotot_jointax', header=[0,1,2,3,4,5,6,7])
# Drop 2 first columns
Jointax = Jointax.drop(Jointax.columns[0:2], axis=1)

In [5]:
# Read the Excel file for the checked genera
selected = pd.read_excel("/home/beatriz/MIC/2_Micro/data/finalist_dfs.xlsx", sheet_name='checked_genera', header=[0,1,2,3,4,5,6,7])
# Drop first row specifically (index 0 which contains NaNs)
selected = selected.drop(index=0)
# Drop first column (the index column with Level1, Level2, etc)
selected = selected.drop(selected.columns[0:3], axis=1)

In [6]:
selected_list = selected.columns.get_level_values(6)

In [13]:
# Extract Genera and ID from the multi-index, For selected genera
selected_GID = dict(zip(selected.columns.get_level_values(6), selected.columns.get_level_values(7)))
# For all genera 
all_GID = dict(zip(Jointax.columns.get_level_values(6), Jointax.columns.get_level_values(7)))

# 3. Reference Formating Function
Following function is to take the references given in the search and present them on APA style list

In [20]:
def format_apa_reference(article):
    """Format article data into APA style reference"""
    try:
        # Get authors
        if 'AuthorList' in article:
            authors = article['AuthorList']
            if len(authors) > 6:
                author_text = f"{authors[0]['LastName']}, {authors[0].get('ForeName', '')[0]}., et al."
            else:
                author_list = []
                for author in authors:
                    if 'ForeName' in author:
                        author_list.append(f"{author['LastName']}, {author['ForeName'][0]}.")
                    else:
                        author_list.append(f"{author['LastName']}")
                author_text = ", ".join(author_list[:-1]) + " & " + author_list[-1] if len(author_list) > 1 else author_list[0]
        else:
            author_text = "No author"

        # Get year
        pub_date = article['Journal']['JournalIssue']['PubDate']
        year = pub_date.get('Year', 'n.d.')

        # Get title
        title = article.get('ArticleTitle', 'No title')
        
        # Get journal info
        journal = article['Journal']
        journal_title = journal.get('Title', journal.get('ISOAbbreviation', 'No journal'))
        
        # Get volume, issue, pages
        volume = journal['JournalIssue'].get('Volume', '')
        issue = journal['JournalIssue'].get('Issue', '')
        pagination = article.get('Pagination', {}).get('MedlinePgn', '')

        # Format the reference
        reference = f"{author_text} ({year}). {title}. {journal_title}"
        if volume:
            reference += f", {volume}"
        if issue:
            reference += f"({issue})"
        if pagination:
            reference += f", {pagination}"
        reference += "."

        return reference
    except Exception as e:
        return f"Error formatting reference: {str(e)}"

# 4. Query DB: Searching Corrosion Genes
This function search on PubMed database the bacteria in the list for different criteria related to corrosion, in order to found which of the bacteria has been previouly identified as causing damage by corrosion. The funciton search various terms used in corrosion and metabolic pathways, then the literature_analysis is done.

In [21]:
def search_corrosion_genes(bacteria_name, base_dir, gid_dict):
    """
This function searches for bacteria's involvement in corrosion processes through:
Literature search using specific corrosion-related terms
Analysis of metabolic pathways and genes related to corrosion

    Parameters:
    bacteria_name: str - name of the bacteria to search
    base_dir: Path - directory where the Excel file will be stored containing:
        - Main Analysis sheet: Complete table with columns Name, Metabolism, Hits etc
        - References sheet: Citations in APA format
        - Abstracts sheet: Full paper abstracts
    gid_dict: dict - mapping of bacteria names to their GIDs
    """
    # Get GID for this bacteria, so that we can identify with name and ID
    bacteria_gid = gid_dict.get(bacteria_name, f"NEW_{bacteria_name}")  # Use NEW_ prefix for new bacteria
    # Create a timestamped filename for this run
    # timestamp = datetime.now().strftime('%Y%m%d_%H%M')   
    # Defining the results file within base_dir
    results_file = base_dir / f"bacteria_corrosion_summary.xlsx"
   
    # Add timing for individual bacteria
    bacteria_start_time = time.time()
    print(f"Starting search for {bacteria_name} at: {datetime.now().strftime('%H:%M:%S')}")
    
    results = {
        'bacteria': bacteria_name,
        'sulfate_reduction': False,
        'metal_reduction': False,
        'corrosion_associated': False,
        'cytochrome_c3': False,
        'acid_production': False,
        'biofilm_formation': False,
        'h2s_production': False,
        'literature_count': 0,
        'evidence': [],
        'processing_time': 0,
    }
    
    # Creating a structured record for each bacteria
    bacteria_record = {
        'Name': bacteria_name,      # Bacteria species/strain name
        'Metabolism': [],          # List of identified metabolic pathways
        'Terms': [],              # Search terms that yielded results
        'Hits': 0,               # Total number of relevant papers found
        'Best_Reference': '',    # Most relevant paper in APA format
        'Abstract': ''          # Abstract from key paper
    }
    
    try:
        # 1. Check KEGG for pathways and genes
        base_url = "http://rest.kegg.jp/" # Check for metabolic pathways and gene presence. Functional anotation.

    except Exception as e:
        print(f"KEGG API error for {bacteria_name}: {str(e)}")
        
        # retry strategy in case there is connectivity issues
        retry_strategy = Retry(
            total=3,
            backoff_factor=1,
            status_forcelist=[500, 502, 503, 504]
        )
        adapter = HTTPAdapter(max_retries=retry_strategy)
        session = requests.Session()
        session.mount("http://", adapter)
        session.mount("https://", adapter)
        # Using sessions no request
        pathway_response = session.get(f"{base_url}find/module/{bacteria_name}")
        pathway_text = pathway_response.text.lower()
        
        # Define search terms key metabolic processes related to corrosion
        sulfate_terms = [
                'sulfate', 'sulphate',    # Terms related to sulfate reduction pathway
                'dsrab', 'dsra', 'dsrb',  # Key genes in dissimilatory sulfate reduction
                'aprab', 'apra', 'aprb',  # Adenosine-5'-phosphosulfate reductase genes
                'sat',  # Sulfate adenylyltransferase
                'sox',  # Sulfur oxidation
                'sir',  # Sulfite reductase
                'aps'   # Adenosine phosphosulfate
        ]                     
        metal_terms = [
                'metal', 'iron', 'fe(iii)', 'metal deterioration', 'MIC',
                'cytochrome', 'corrosion', 'biocorrosion',
                'methane corrosion', 'methanogenesis corrosion',
                'bacteria corrosion', 'anaerobic corrosion',
                'biofilm corrosion', 'manganese corrosion',
                'denitrification corrosion',
                'mtr',  # Metal reduction
                'omc',  # Outer membrane cytochromes
                'pil',  # Pili genes involved in metal reduction
                'cymA',  # Cytoplasmic membrane protein
                'hydA',  # Hydrogenase
                'feo',  # Ferrous iron transport
                'nrf',   # Nitrite reduction
                'organic acid AND corrosion',
                'acid metabolite AND metal deterioration',
                'fermentation AND corrosion',
                'biofilm AND (corrosion OR MIC)',
                'hydrogen sulfide AND corrosion',
                'thiosulfate AND corrosion'
                ]

        # Check pathway text
        if any(term in pathway_text for term in sulfate_terms):
            # Verify pathway presence
            pathway_evidence = [term for term in sulfate_terms if term in pathway_text]
            if pathway_evidence:
                results['sulfate_reduction'] = True
                results['evidence'].append(f"Found sulfate pathway evidence: {pathway_evidence}")
            else:
                # Flag potential inconsistency
                print(f"Warning: {bacteria_name} shows sulfate reduction hits but no clear pathway evidence")
        
        if any(term in pathway_text for term in metal_terms):
            results['metal_reduction'] = True
            results['evidence'].append(f"Found metal pathway: {[term for term in metal_terms if term in pathway_text]}")
        
        # Look for genes
        genes_response = session.get(f"{base_url}find/genes/{bacteria_name}")
        genes_text = genes_response.text.lower()
        
        if "cytochrome c3" in genes_text:
            results['cytochrome_c3'] = True
            results['evidence'].append("Found cytochrome c3 gene")
        
        if any(gene in genes_text for gene in ['dsr', 'apr', 'sat']):
            results['sulfate_reduction'] = True
            results['evidence'].append(f"Found sulfate genes: {[gene for gene in ['dsr', 'apr', 'sat'] if gene in genes_text]}")
            
    except Exception as e:
        print(f"KEGG API error for {bacteria_name}: {str(e)}")
    
    # 3. Check literature
    try:
        Entrez.email = "beatrizamandawatts@gmail.com"
             
        search_terms = [
            f"{bacteria_name}[Organism] AND corrosion[Title]",
            f"{bacteria_name}[Organism] AND biocorrosion[Title]",
            f"{bacteria_name}[Organism] AND 'microbiologically influenced corrosion'[Title]",
            f"{bacteria_name}[Organism] AND (dsrAB OR aprAB) AND corrosion", # sulphate metabolism
            f"{bacteria_name}[Organism] AND (metal reduction OR iron reduction)",   # metal interaction
            f"{bacteria_name}[Organism] AND (cytochrome c3) AND corrosion",
            f"{bacteria_name}[Organism] AND corrosion",
            f"{bacteria_name}[Organism] AND biocorrosion",
            f"{bacteria_name}[Organism] AND (MIC OR 'microbiologically influenced corrosion')",
            f"{bacteria_name}[Organism] AND 'material deterioration'",
            f"{bacteria_name}[Organism] AND ('metal deterioration' OR 'metallic corrosion')",
            f"{bacteria_name}[Organism] AND (acid production) AND (corrosion OR 'metal deterioration' OR MIC)",
            f"{bacteria_name}[Organism] AND AND biofilm AND (corrosion OR MIC)",
            f"{bacteria_name}[Organism] AND (ochre formation OR iron oxide deposits OR rust formation)",
            f"{bacteria_name}[Organism] AND (hydrogen sulfide OR H2S) AND (corrosion OR 'metal deterioration')"
            f"{bacteria_name}[Organism] AND ('sulfate reducing bacteria'[Title/Abstract] AND corrosion)",
            f"{bacteria_name}[Organism] AND ('metal reducing bacteria'[Title/Abstract] AND corrosion)",
            ]
        
        for term in search_terms:
            handle = Entrez.esearch(db="pubmed", term=term)
            record = Entrez.read(handle)
            count = int(record["Count"])
            results['literature_count'] += count
            
            if count > 0:
                results['evidence'].append(f"Found {count} papers for: {term}")
                paper_ids = record["IdList"]
                
                try:
                    papers_handle = Entrez.efetch(db="pubmed", id=paper_ids, rettype="medline", retmode="xml")
                    papers = Entrez.read(papers_handle)
                    print(f"Found {len(papers.get('PubmedArticle', []))} papers for {bacteria_name}")  # Debug line
                    
                    # Update metabolism flags
                    if "sulfate" in term.lower():
                        results['sulfate_reduction'] = True
                        if 'Sulfate Reduction' not in bacteria_record['Metabolism']:
                            bacteria_record['Metabolism'].append('Sulfate Reduction')
                    
                    if "metal" in term.lower():
                        results['metal_reduction'] = True
                        if 'Metal Reduction' not in bacteria_record['Metabolism']:
                            bacteria_record['Metabolism'].append('Metal Reduction')
                    
                    if "cytochrome" in term.lower():
                        results['cytochrome_c3'] = True
                        if 'Cytochrome c3' not in bacteria_record['Metabolism']:
                            bacteria_record['Metabolism'].append('Cytochrome c3')
                    
                    bacteria_record['Hits'] += count
                    bacteria_record['Terms'].append(f"{term}: {count} hits")

                    if papers.get('PubmedArticle'):
                        latest_paper = papers['PubmedArticle'][0]
                        article = latest_paper['MedlineCitation']['Article']
                    
                    # Store reference if it's corrosion-related (broadened criteria)
                    if ('corrosion' in article['ArticleTitle'].lower() or 
                        'mic' in article['ArticleTitle'].lower() or
                        'metal' in article['ArticleTitle'].lower()):
                        bacteria_record['Best_Reference'] = format_apa_reference(article)
                        if 'Abstract' in article:
                            bacteria_record['Abstract'] = article['Abstract']['AbstractText'][0]

                    time.sleep(1)  # Being nice to the APIs
                    
                except Exception as e:
                    print(f"Error processing PubMed data for {bacteria_name}: {e}")       
                    # Add debug print before saving
                    print(f"Saving reference for {bacteria_name}: {bacteria_record.get('Best_Reference', 'No reference')}")
        # Save to Excel
        try:           
            # Load existing data for this run or create new DataFrame
            if results_file.exists():
                main_df = pd.read_excel(results_file, sheet_name='Analysis', index_col=0)
                refs_df= pd.read_excel(results_file, sheet_name ='References_Abstracts', index_col=0)
            else:
                # First bacteria in this tun creates new df
                main_df = pd.DataFrame(columns=['Name', 'Metabolism', 'Terms', 'Hits'],
                                            index=pd.Index([], name='GID'))
                refs_df=pd.DataFrame(columns=['Name', 'Reference', 'Abstract'],    
                                            index=pd.Index([], name='GID'))                      
            # Prepare new row for main sheet
            new_row = pd.DataFrame({
                'Name': [bacteria_name],
                'Metabolism': ['; '.join(bacteria_record['Metabolism']) if bacteria_record['Metabolism'] else ''],
                'Terms': ['; '.join(bacteria_record['Terms']) if bacteria_record['Terms'] else ''],
                'Hits': [bacteria_record['Hits']]
            }, index=[bacteria_gid])

            # New row for ref sheet
            new_refs_row = pd.DataFrame({
                'Name': [bacteria_name],
                'Reference': [bacteria_record.get('Best_Reference', '')],
                'Abstract': [bacteria_record.get('Abstract', '')]
             }, index=[bacteria_gid])

            # Update or append to DataFrames
            if bacteria_gid in main_df.index:
                main_df.loc[bacteria_gid] = new_row.iloc[0]
            else:
                main_df = pd.concat([main_df, new_row])
            # Update or append to DataFrames
            if bacteria_gid in refs_df.index:
                refs_df.loc[bacteria_gid] = new_refs_row.iloc[0]
            else:
                refs_df = pd.concat([refs_df, new_refs_row])    
                                    
            # Save both DataFrames to the same file
            with pd.ExcelWriter(results_file, engine='openpyxl', mode='w') as writer:
                main_df.to_excel(writer, sheet_name='Analysis')
                refs_df.to_excel(writer, sheet_name='References_Abstracts')
                
                # Format columns
                for sheet in writer.sheets.values():
                    sheet.column_dimensions['B'].width = 30
                    if sheet.title == 'References_Abstracts':
                        sheet.column_dimensions['C'].width = 50
                        sheet.column_dimensions['D'].width = 50
                        for row in sheet.iter_rows(min_row=2, min_col=3, max_col=4):
                            for cell in row:
                                cell.alignment = openpyxl.styles.Alignment(wrap_text=True)
        except Exception as e:
            print(f"Error saving to Excel for {bacteria_name}: {str(e)}")

    except Exception as e:
        print(f"Error in literature processing for {bacteria_name}: {e}")

    finally:
        results['processing_time'] = time.time() - bacteria_start_time
        print(f"Finished {bacteria_name} in {results['processing_time']:.2f} seconds")

        return results

In [ ]:
# For your selected bacteria
for bacteria in selected_list:
    result = search_corrosion_genes(bacteria, base_dir, selected_GID)
    print(f"\nResults for {bacteria}:")
    print(f"Sulfate reduction: {result['sulfate_reduction']}")
    print(f"Metal reduction: {result['metal_reduction']}")
    print(f"Cytochrome c3: {result['cytochrome_c3']}")
    print(f"Literature count: {result['literature_count']}")
    print("Evidence:", "\n- ".join([''] + result['evidence']))
gc.collect()

Starting search for Anaerococcus at: 02:14:56
Found 7 papers for Anaerococcus
Finished Anaerococcus in 11.33 seconds

Results for Anaerococcus:
Sulfate reduction: False
Metal reduction: False
Cytochrome c3: False
Literature count: 7
Evidence: 
- Found 7 papers for: Anaerococcus[Organism] AND (MIC OR 'microbiologically influenced corrosion')
Starting search for Aquamicrobium at: 02:15:07
Finished Aquamicrobium in 13.28 seconds

Results for Aquamicrobium:
Sulfate reduction: False
Metal reduction: False
Cytochrome c3: False
Literature count: 0
Evidence: 
Starting search for Azospira at: 02:15:20
Found 20 papers for Azospira
Found 1 papers for Azospira
Finished Azospira in 16.75 seconds

Results for Azospira:
Sulfate reduction: False
Metal reduction: True
Cytochrome c3: False
Literature count: 23
Evidence: 
- Found 22 papers for: Azospira[Organism] AND (metal reduction OR iron reduction)
- Found 1 papers for: Azospira[Organism] AND biocorrosion
Starting search for Brachybacterium at: 02:15

# 6. Analysis of Search Results
The literature search validates our statistical selection of significant bacteria. Most of these 30 genera, chosen from 882 bacteria and archaea based on statistical significance, show evidence of corrosion-related activity in existing literature.
The results demonstrate varying levels of prior documentation:

Well-documented corrosion-causers (e.g., Thiobacillus, Streptococcus): These serve as positive controls, confirming our statistical approach
Moderately documented genera: Support our findings while suggesting areas for further research
Novel candidates with minimal documentation (e.g., Bulleidia, Mycoplana, Oxobacter): These represent potentially new corrosion-associated bacteria identified through our statistical analysis

The presence of well-known corrosion-causing bacteria in our statistically significant set validates our analytical approach. This gives more weight to our novel findings regarding the less-studied bacteria in our selection.
Next steps with PICRUSt functional analysis will help understand the metabolic capabilities of our newly identified bacteria, using the well-documented corrosion-causing bacteria as reference points for comparison.

Some improvement was done in the following function. The next function more tailored to corrosion influencing bacteria, which title has any word on corrosion exactly.

Prioritizing papers with "corrosion" in the title. Focusing on heating/cooling system contexts
Weighting hits based on relevance to industrial systems.

# 7. Improved function series

In [14]:
def search_pubmed_improved(bacteria_name, search_terms):
    """
    Performs improved PubMed search with structured results
    """
    Entrez.email = "beatrizamandawatts@gmail.com"  
    results = {
        'primary_hits': [], 
        'secondary_hits': [],
        'context_hits': []
    }
    
    # Search each category
    for category in search_terms:
        for term in search_terms[category]:
            try:
                handle = Entrez.esearch(db="pubmed", term=term)
                record = Entrez.read(handle)
                if int(record["Count"]) > 0:
                    paper_ids = record["IdList"]
                    papers_handle = Entrez.efetch(db="pubmed", id=paper_ids, 
                                                rettype="medline", retmode="xml")
                    papers = Entrez.read(papers_handle)
                    results[f'{category}_hits'].extend(papers.get('PubmedArticle', []))
                time.sleep(1)
            except Exception as e:
                print(f"Error searching {term}: {str(e)}")
                
    return results

In [15]:
def analyze_kegg_pathways(bacteria_name):
    """
    Analyzes KEGG pathways for corrosion-relevant processes
    """
    base_url = "http://rest.kegg.jp/"
    pathways = {
        'sulfate_reduction': [],
        'metal_reduction': [],
        'biofilm_formation': []
    }
    
    try:
        # Look for pathway modules
        pathway_response = requests.get(f"{base_url}find/module/{bacteria_name}")
        pathway_text = pathway_response.text.lower()
        
        # Check for relevant pathways
        if any(term in pathway_text for term in ['sulfate', 'dsrab', 'aprab']):
            pathways['sulfate_reduction'].append('sulfate_reduction_pathway')
            
        if any(term in pathway_text for term in ['metal', 'iron', 'fe(iii)']):
            pathways['metal_reduction'].append('metal_reduction_pathway')
            
        if 'biofilm' in pathway_text:
            pathways['biofilm_formation'].append('biofilm_formation_pathway')
            
    except Exception as e:
        print(f"KEGG API error for {bacteria_name}: {str(e)}")
        
    return pathways

def calculate_evidence_score(pubmed_results, kegg_pathways, weights):
    """
    Calculates weighted evidence score
    """
    score = 0
    
    # Score from primary literature hits
    score += len(pubmed_results['primary_hits']) * weights['title_match']
    
    # Score from secondary hits
    score += len(pubmed_results['secondary_hits']) * 1.0
    
    # Score from context hits
    score += len(pubmed_results['context_hits']) * weights['context_match']
    
    # Score from KEGG pathways
    pathway_count = sum(len(pathways) for pathways in kegg_pathways.values())
    score += pathway_count * 0.5
    
    return score

In [16]:
def identify_mechanisms(pubmed_results, kegg_pathways):
    """
    Identifies corrosion mechanisms based on literature and pathway evidence
    """
    mechanisms = []
    
    # Check literature evidence
    if pubmed_results['primary_hits']:
        mechanisms.extend(['direct_corrosion_evidence'])
    
    # Check pathway evidence
    for pathway_type, pathways in kegg_pathways.items():
        if pathways:
            mechanisms.append(pathway_type)
    
    return list(set(mechanisms))  # Remove duplicates

In [17]:
def calculate_final_results(results, weights):
    # Calculate total evidence score
    total_score = (
        len(results['primary_hits']) * weights['title_match'] +
        len(results['secondary_hits']) * 1.0 +
        len(results['context_hits']) * weights['context_match'] +
        (results['total_citations'] * weights['citation_impact']) +
        (len(results['metabolism_pathways']) * weights['pathway_evidence'])
    )
    
    # Update confidence level
    results['confidence_level'] = determine_confidence_level(total_score)
    
    # Deduplicate evidence types and environments
    results['evidence_types'] = list(set(results['evidence_types']))
    results['environment_mentions'] = list(set(results['environment_mentions']))
    
    return results

In [18]:
def search_corrosion_genes_improved(bacteria_name, base_dir, gid_dict):
    """
    Enhanced version of search_corrosion_genes with improved specificity and filtering
    """
    Entrez.email = "beatrizamandawatts@gmail.com"  
    results = {
        'bacteria': bacteria_name,
        'primary_hits': [], 
        'secondary_hits': [],
        'context_hits': [],
        'experimental_evidence': False,
        'confidence_level': 'low',    # updated based on evidence
        'evidence_types': [],         # direct/indirect/environmental
        'total_citations': 0,
        'recent_papers': 0,          # papers from last 5 years
        'metabolism_pathways': [],
        'environment_mentions': []
    }
    # Add evidence scoring weights
    weights = {
        'title_match': 2.0,
        'experimental': 3.0,
        'recent_paper': 1.2,
        'citation_impact': 0.1,
        'context_match': 1.5,
        'pathway_evidence': 1.0
    }
    results_file = base_dir / "bacteria_corrosion_summary_improved.xlsx"
    bacteria_gid = gid_dict.get(bacteria_name, f"NEW_{bacteria_name}")

    # Refined search terms with weights
    search_terms = {
        'primary': [
            f'"{bacteria_name}"[Organism] AND "microbiologically influenced corrosion"[Title]',
            f'"{bacteria_name}"[Organism] AND "biocorrosion"[Title]',
            f'"{bacteria_name}"[Organism] AND "metal corrosion"[Title]',
            f'"{bacteria_name}"[Organism] AND "deterioration"[Title]',
            f'"{bacteria_name}"[Organism] AND "metal oxidation"[Title]',
            f'"{bacteria_name}"[Organism] AND "metallic corrosion"[Title]',
            f'"{bacteria_name}"[Organism] AND "experimental"[Title/Abstract] AND "corrosion"[Title]',
            f'"{bacteria_name}"[Organism] AND "mechanism"[Title/Abstract] AND "corrosion"[Title]'
        ],
        'secondary': [
            f'"{bacteria_name}"[Organism] AND (dsrAB[Title/Abstract] OR aprAB[Title/Abstract])',
            f'"{bacteria_name}"[Organism] AND "sulfate reducing"[Title]',
            f'"{bacteria_name}"[Organism] AND "metal reduction"[Title]',
            f'"{bacteria_name}"[Organism] AND "electron transfer"[Title/Abstract]',
            f'"{bacteria_name}"[Organism] AND "biofilm formation"[Title/Abstract]',
            f'"{bacteria_name}"[Organism] AND "cytochrome c3"[Title/Abstract]',
            f'"{bacteria_name}"[Organism] AND (sat OR sox OR sir OR aps)[Title/Abstract]',
            f'"{bacteria_name}"[Organism] AND (mtr OR omc OR pil OR cymA)[Title/Abstract]',
            f'"{bacteria_name}"[Organism] AND "electron transfer"[Title/Abstract]',
            f'"{bacteria_name}"[Organism] AND "metabolic pathway"[Title/Abstract]'
        ],
        'context': [
            'heating system',
            'cooling system',
            'industrial water',
            'pipeline corrosion',
            'metal infrastructure',
            'industrial biotechnology',
            'water treatment',
            'oil pipeline',        
            'biofilm AND (corrosion OR MIC)',
            'hydrogen sulfide AND corrosion',
            'acid production AND (corrosion OR metal)',
            'industrial environment',
            'metal infrastructure',
            'microbial community'
        ]
    }
       
    try:
        # KEGG pathway analysis
        kegg_pathways = analyze_kegg_pathways(bacteria_name)
        
        # PubMed literature search with improved filtering
        pubmed_results = search_pubmed_improved(bacteria_name, search_terms)
        
        # Score calculation
        evidence_score = calculate_evidence_score(pubmed_results, kegg_pathways, weights)
        
        # Store results
        results['evidence_quality'] = evidence_score
        results['mechanisms'] = identify_mechanisms(pubmed_results, kegg_pathways)
    
        # Save results to Excel
        try:
            # Load existing data or create new DataFrames
            if results_file.exists():
                main_df = pd.read_excel(results_file, sheet_name='Analysis', index_col=0)
                refs_df = pd.read_excel(results_file, sheet_name='References_Abstracts', index_col=0)
            else:
                main_df = pd.DataFrame(columns=['Name', 'Mechanisms', 'Evidence_Quality', 'Total_Hits'],
                                     index=pd.Index([], name='GID'))
                refs_df = pd.DataFrame(columns=['Name', 'Reference', 'Abstract'],
                                     index=pd.Index([], name='GID'))

            # Prepare new row for main sheet
            new_row = pd.DataFrame({
                'Name': [bacteria_name],
                'Confidence_Level': [results['confidence_level']],
                'Evidence_Types': ['; '.join(results['evidence_types'])],
                'Total_Citations': [results['total_citations']],
                'Experimental_Evidence': [results['experimental_evidence']],
                'Environment_Context': ['; '.join(results['environment_mentions'])],
                'Mechanisms': ['; '.join(results['metabolism_pathways'])]
            }, index=[bacteria_gid])

            # Get the best corrosion-related paper
            best_paper = None
            if pubmed_results['primary_hits']:
                best_paper = pubmed_results['primary_hits'][0]
            elif pubmed_results['secondary_hits']:
                best_paper = pubmed_results['secondary_hits'][0]

            if best_paper:
                article = best_paper['MedlineCitation']['Article']
                new_refs_row = pd.DataFrame({
                    'Name': [bacteria_name],
                    'Reference': [format_apa_reference(article)],
                    'Abstract': [article.get('Abstract', {}).get('AbstractText', [''])[0]]
                }, index=[bacteria_gid])
            else:
                new_refs_row = pd.DataFrame({
                    'Name': [bacteria_name],
                    'Reference': [''],
                    'Abstract': ['']
                }, index=[bacteria_gid])

            # Update or append to DataFrames
            if bacteria_gid in main_df.index:
                main_df.loc[bacteria_gid] = new_row.iloc[0]
            else:
                main_df = pd.concat([main_df, new_row])

            if bacteria_gid in refs_df.index:
                refs_df.loc[bacteria_gid] = new_refs_row.iloc[0]
            else:
                refs_df = pd.concat([refs_df, new_refs_row])

            # Save both DataFrames
            with pd.ExcelWriter(results_file, engine='openpyxl', mode='w') as writer:
                main_df.to_excel(writer, sheet_name='Analysis')
                refs_df.to_excel(writer, sheet_name='References_Abstracts')

                # Format columns
                for sheet in writer.sheets.values():
                    sheet.column_dimensions['B'].width = 30
                    if sheet.title == 'References_Abstracts':
                        sheet.column_dimensions['C'].width = 50
                        sheet.column_dimensions['D'].width = 50
                        for row in sheet.iter_rows(min_row=2, min_col=3, max_col=4):
                            for cell in row:
                                cell.alignment = openpyxl.styles.Alignment(wrap_text=True)

        except Exception as e:
            print(f"Error saving to Excel for {bacteria_name}: {e}")
            
    except Exception as e:
        logging.error(f"Error processing {bacteria_name}: {str(e)}")
        return None
        
    return results

In [19]:
# For selected bacteria
for bacteria in selected_list:
    result = search_corrosion_genes_improved(bacteria, base_dir, selected_GID) 
    print(f"\nResults for {bacteria}:")
    print(f"Evidence quality score: {result['evidence_quality']}")
    print(f"Identified mechanisms: {', '.join(result['mechanisms'])}")


Error saving to Excel for Anaerococcus: name 'format_apa_reference' is not defined

Results for Anaerococcus:
Evidence quality score: 424.0
Identified mechanisms: 
Error saving to Excel for Aquamicrobium: name 'format_apa_reference' is not defined

Results for Aquamicrobium:
Evidence quality score: 422.0
Identified mechanisms: 
Error saving to Excel for Azospira: name 'format_apa_reference' is not defined

Results for Azospira:
Evidence quality score: 442.0
Identified mechanisms: 
Error saving to Excel for Brachybacterium: name 'format_apa_reference' is not defined

Results for Brachybacterium:
Evidence quality score: 422.0
Identified mechanisms: 
Error saving to Excel for Brevibacterium: name 'format_apa_reference' is not defined

Results for Brevibacterium:
Evidence quality score: 452.0
Identified mechanisms: direct_corrosion_evidence
Error saving to Excel for Bulleidia: name 'format_apa_reference' is not defined

Results for Bulleidia:
Evidence quality score: 421.0
Identified mechan

KeyboardInterrupt: 

# Network analysis

Ecological Networks:


Bacteria that appear "neutral" alone might be critical support species
They could be enabling or moderating the effects of the corrosion-significant species
In microbial communities, some species act as "keystone" species not through abundance but through their metabolic interactions


Stability Indicators:


Species present across all conditions might be:

Buffer species that maintain community stability
Indicators of baseline environmental conditions
Part of the core microbiome that enables other species to thrive

Think of it like a metal alloy - some elements might not directly affect corrosion resistance, but their presence maintains the overall structure that makes the protective elements effective.
However, if data size/processing is a significant concern, you could:

Keep full bacterial data initially
Run your analysis
Check if removing the "uniform" species significantly changes your results
Document which removals affect the model and which don't
_________________________
This is to understand genus interactions
Group bacteria by their typical ecological roles (e.g., primary degraders, secondary degraders)
Add known syntrophic relationships between genera
Map carbon/nitrogen cycling capabilities
Identify potential metabolic handoffs between community members
__

Map each genus to known electron acceptor preferences (Fe, Mn, S, etc.)
Create functional groups based on these metabolic capabilities
Compare distribution of these functional groups across your categories
Look for enrichment patterns of specific metabolic types


# QIIME2 (Microbiome analysis)

In [34]:
# Import FASTA into QIIME 2
qiime tools import \
  --input-path your_sequences.fasta \
  --output-path sequences.qza \
  --type 'FeatureData[Sequence]'

# Run DADA2 or Deblur for ASV generation
qiime dada2 denoise-single \
  --i-demultiplexed-seqs sequences.qza \
  --p-trim-left 0 \
  --p-trunc-len 250 \
  --o-representative-sequences rep-seqs.qza \
  --o-table table.qza

# Export to BIOM format
qiime tools export \
  --input-path table.qza \
  --output-path exported-table

# Convert to TSV if needed
biom convert \
  -i exported-table/feature-table.biom \
  -o feature-table.tsv \
  --to-tsv


# Dereplicate sequences
vsearch --derep_fulllength your_sequences.fasta \
        --output unique_sequences.fasta \
        --sizeout

# Cluster at 97% similarity (for OTUs)
vsearch --cluster_size unique_sequences.fasta \
        --id 0.97 \
        --centroids clustered_sequences.fasta

# Create OTU table
vsearch --usearch_global your_sequences.fasta \
        --db clustered_sequences.fasta \
        --id 0.97 \
        --otutabout otu_table.txt



SyntaxError: invalid syntax (51958122.py, line 2)